In [ ]:
from itertools import product

import numpy as np
import pandas as pd

from matplotlib.pylab import plt
import seaborn as sns

from luescher_nd.zeta.zeta3d import Zeta3D
import luescher_nd.utilities as ut

from luescher_nd.utilities import get_laplace_coefficients
from luescher_nd.utilities import Solver

In [ ]:
epsilon = 0.1
L = 1.0
n1d_max = int(L / epsilon)
nstep=2

HBARC = 197.326 # MeV / fm
M_NUCLEON = (938.27 + 939.57) / 2 / HBARC # in fm^{-1}

mu = M_NUCLEON / 2

zeta3d = Zeta3D(L, epsilon)

In [ ]:
solver = ut.Solver(
    n1d_max=n1d_max,
    ndim_max=3,
    lattice_spacing=epsilon,
    derivative_shifts=ut.get_laplace_coefficients(nstep),
)

In [ ]:
c0 = -5.0
energies = solver.get_energies(c0, n_energies=2)
all_x = energies * mu * L ** 2 / 2 / np.pi ** 2

In [ ]:
all_x

In [ ]:
def effective_range(x, L, epsilon):
    """
    """
    p2 = x / (L / 2 / np.pi) ** 2
    Lambda = np.pi / epsilon
    r0 = 4 / np.pi / Lambda
    P = -np.pi ** 2 / 96
    return r0 / 2 * p2 - P * r0 ** 3 * p2 ** 2

In [ ]:
df = pd.read_csv("../data/luescher-3d-res-eff.csv")
df["L"] = df["n1d_max"]  * df["epsilon"]
df["x"] = df["energy"] * mu * df["L"]**2 / 2 / np.pi**2

df["y"] = None
for L, epsilon in product(df.L.unique(), df.epsilon.unique()):
    ind = (df.L == L) & (df.epsilon == epsilon)
    zeta = Zeta3D(L, epsilon)
    df.loc[ind, "y"] = zeta(df.loc[ind, "x"].values) / np.pi / L
    df.loc[ind, "y_eff"] = df.loc[ind, "y"] + effective_range(df.loc[ind, "x"].values, L, epsilon)
    

df.head()

In [ ]:
tf = df[(df.y.abs() < 10) & (df.nstep > 1)].drop_duplicates(["epsilon", "nstep", "y"])

In [ ]:
ymax = 4
g = sns.FacetGrid(
    tf,
    col="nstep",
    hue="n1d_max",
    hue_kws={"marker": ["s", "o", "x", "d"]},
    palette="cubehelix",
    xlim=(-1, 6),
    ylim=(-ymax, ymax)
)

g.map(plt.plot, "x", "y", ms=1, lw=0.5, ls="--")

g.set_xlabels("$x$")
g.set_ylabels(r"$\frac{S_3^{Lat}(x)}{\pi L}$")

g.add_legend(title=r"$\frac{L}{\epsilon}$", fontsize=6, bbox_to_anchor=(1.1, 0.65))

x = np.linspace(-1, 10, 1000)

for ax in g.axes[0]:
    ax.axhline(0, c="black", lw=0.2)
    ax.axvline(0, c="black", lw=0.2)
    ax.plot(x, zeta3d(x)/np.pi/L, "--", lw=0.1, ms=0.1)
    
g.fig.set_dpi(300)
g.fig.set_figheight(2)
g.fig.set_figwidth(6)

plt.show(g.fig)

In [ ]:
g.fig.savefig("S3lat-results.pdf", bbox_inches="tight")

In [ ]:
!open S3lat-results.pdf